# Copy an analysis job from a Flywheel Instance

Given the ID of a Flywheel job, this will create a python script to re-run it.  You can then edit and run that script.

The cell below will get a Flywheel client if you are logged in to a Flywheel instance.  It prints out the URL of the instance so you know where you are logged in.

In [1]:
import argparse
import os
import pprint
import stat

import flywheel

In [2]:
fw = flywheel.Client("")
print("Flywheel Instance", fw.get_config().site.api_url)

/usr/local/lib/python3.9/site-packages/flywheel/flywheel.py:6274: UserWarning: Client version 15.3.1 does not match server version 14.6.2. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


Flywheel Instance https://rollout.ce.flywheel.io/api


Get the job ID from the URL when you select the job of interest in the "Jobs Log" in the Flywheel UI.

Create a python script to re-run a job given the job ID for a gear that was run on Flywheel.

In [24]:
def write_script_to_run_job(job_id=None, analysis_id=None):

    analysis = None
    if analysis_id:
        analysis = fw.get_analysis(analysis_id)
        print(f"Getting job_id from analysis '{analysis.label}'")
        job_id = analysis.job.id
    elif job_id == None:
        print(f"Must provide either job_id or analysis_id.")
        os.sys.exit(1)

    print("Job ID", job_id)
    job = fw.get_job(job_id)
    gear = fw.get_gear(job.gear_id)
    print(f"gear.gear.name is {gear.gear.name}")
    destination_id = job.destination.id
    destination_type = job.destination.type
    print(f"job's destination_id is {destination_id} type {destination_type}")

    if job.destination.type == "analysis":
        analysis = fw.get_analysis(destination_id)
        destination_id = analysis.parent.id
        destination_type = analysis.parent.type
        print(f"job's analysis's parent id is {destination_id} type {destination_type}")

    destination = fw.get(destination_id)
    destination_label = destination.label
    print(f"new job's destination is {destination_label} type {destination_type}")

    group_id = destination.parents.group
    print(f"Group id: {group_id}")

    if destination_type == "project":
        project = destination
    else:
        project = fw.get_project(destination.parents.project)
    project_label = project.label
    print(f"Project label: {project.label}")

    script_name = f"{project_label}_{destination_type}_{destination.label}.py"
    script_name = script_name.replace(" ", "_")

    container_path = "Invalid"

    if destination_type == "project":
        container_path = f"{group_id}/{project_label}"

    elif destination_type == "subject":
        container_path = f"{group_id}/{project_label}/{destination.label}"

    elif destination_type == "session":
        container_path = (
            f"{group_id}/{project_label}/{destination.subject.label}/"
            + f"{destination.label}"
        )

    elif destination_type == "acquisition":
        subject = fw.get_subject(destination.parents.subject)
        session = fw.get_session(destination.parents.session)
        container_path = (
            f"{group_id}/{project_label}/{subject.label}/{session.label}/"
            + f"{destination.label}"
        )

    else:
        print(f"Error: unknown destination type {destination_type}")

    print(f"container_path: {container_path}")
    print(f"Creating script: {script_name} ...\n")
    
    input_files = dict()
    for key, val in job.config.get("inputs").items():
        if "hierarchy" in val:
            input_files[key] = {
                "hierarchy_id": val["hierarchy"]["id"],
                "location_name": val["location"]["name"],
            }

    lines = f"""#! /usr/bin/env python3
'''Run {gear.gear.name} on {destination_type} "{destination.label}"

    This script was created to run Job ID {job_id}
    In project "{group_id}/{project_label}"
    On Flywheel Instance {fw.get_config().site.api_url}
'''

import os
import argparse
from datetime import datetime


import flywheel


input_files = {pprint.pformat(input_files)}

def main(fw):

    gear = fw.lookup("gears/{gear.gear.name}")
    print("gear.gear.version for job was = {gear.gear.version}")"""

    sfp = open(script_name, "w")
    for line in lines.split("\n"):
        sfp.write(line + "\n")

    sfp.write('    print(f"gear.gear.version now = {gear.gear.version}")\n')
    sfp.write(f'    print("destination_id = {destination_id}")\n')
    sfp.write(f'    print("destination type is: {destination_type}")\n')

    sfp.write(f'    destination = fw.lookup("{container_path}")\n')

    sfp.write("\n")
    sfp.write("    inputs = dict()\n")
    sfp.write("    for key, val in input_files.items():\n")
    sfp.write("         container = fw.get(val['hierarchy_id'])\n")
    sfp.write("         inputs[key] = container.get_file(val['location_name'])\n")
    sfp.write("\n")
    sfp.write(f"    config = {pprint.pformat(job['config']['config'], indent=4)}\n")
    sfp.write("\n")

    if job.destination.type == "analysis":
        sfp.write("    now = datetime.now()\n")
        sfp.write("    analysis_label = (\n")
        sfp.write(
            "        f'{gear.gear.name} {now.strftime(\"%m-%d-%Y %H:%M:%S\")} SDK launched'\n"
        )
        sfp.write("    )\n")
        sfp.write("    print(f'analysis_label = {analysis_label}')\n")

        lines = f"""
    analysis_id = gear.run(
        analysis_label=analysis_label,
        config=config,
        inputs=inputs,
        destination=destination,
    )"""
        for line in lines.split("\n"):
            sfp.write(line + "\n")
        sfp.write("    print(f'analysis_id = {analysis_id}')\n")
        sfp.write("    return analysis_id\n")

    else:
        lines = f"""
    job_id = gear.run(
        config=config,
        inputs=inputs,
        destination=destination
    )"""
        for line in lines.split("\n"):
            sfp.write(line + "\n")
        sfp.write("    print(f'job_id = {job_id}')\n")
        sfp.write("    return job_id\n")

    lines = f"""
if __name__ == '__main__':

    parser = argparse.ArgumentParser(description=__doc__)
    args = parser.parse_args()

    fw = flywheel.Client('')
    print(fw.get_config().site.api_url)

    analysis_id = main(fw)"""

    for line in lines.split("\n"):
        sfp.write(line + "\n")

    sfp.write("\n")
    sfp.write("    os.sys.exit(0)\n")

    sfp.close()

    os.system(f"black {script_name}")

    st = os.stat(script_name)
    os.chmod(script_name, st.st_mode | stat.S_IEXEC)

## Utility Job, destination type is: acquisition

In [29]:
write_script_to_run_job(job_id="60817598fb84816baf6f3572")

Job ID 60817598fb84816baf6f3572
gear.gear.name is dcm2niix
job's destination_id is 60798ec6c5a368b6b66f3475 type acquisition
new job's destination is 4 - fmap-SE-PA type acquisition
Group id: bids-curation-tests
Project label: Levitas_Tutorial
container_path: bids-curation-tests/Levitas_Tutorial/10462@thwjames_OpenScience/2020-01-22 14_29_46/4 - fmap-SE-PA
Creating script: Levitas_Tutorial_acquisition_4_-_fmap-SE-PA.py ...



## Utility Job, destination type is: session

In [25]:
write_script_to_run_job(job_id="60817898f4a3a2bb836f35ca")

Job ID 60817898f4a3a2bb836f35ca
gear.gear.name is curate-bids
job's destination_id is 60798ec5c5a368b6b66f3467 type session
new job's destination is 2020-01-22 14_29_46 type session
Group id: bids-curation-tests
Project label: Levitas_Tutorial
container_path: bids-curation-tests/Levitas_Tutorial/10462@thwjames_OpenScience/2020-01-22 14_29_46
Creating script: Levitas_Tutorial_session_2020-01-22_14_29_46.py ...



In [18]:
gear.run?

Signature:
gear.run(
    config=None,
    analysis_label=None,
    tags=None,
    destination=None,
    inputs=None,
    **kwargs,
)
Docstring:
Run the gear.

:param dict config: The configuration to use, if overriding defaults.
:param str analysis_label: The label of the analysis, if running an analysis gear.
:param list tags: The list of tags to set for the job.
:param object destination: The destination container.
:param dict inputs: The list of input containers or files.

:return: The id of the job that was created
File:      /usr/local/lib/python3.9/site-packages/flywheel/models/gear_mixin.py
Type:      method


## Analysis Job, destination type is: project

In [26]:
write_script_to_run_job(job_id="603fb4ab146a36499c6e8aca")

Job ID 603fb4ab146a36499c6e8aca
gear.gear.name is bids-app-template
job's destination_id is 603fb4ab146a36499c6e8ac9 type analysis
job's analysis's parent id is 602ed7b21c5936816883e901 type project
new job's destination is BIDS_multi_session type project
Group id: scien
Project label: BIDS_multi_session
container_path: scien/BIDS_multi_session
Creating script: BIDS_multi_session_project_BIDS_multi_session.py ...



## Analysis Job, destination type is: session

In [27]:
write_script_to_run_job(job_id="603fb0c775f2cd6a236e8ab5")

Job ID 603fb0c775f2cd6a236e8ab5
gear.gear.name is bids-app-template
job's destination_id is 603fb0c775f2cd6a236e8ab4 type analysis
job's analysis's parent id is 602ed812abe32939b783e910 type session
new job's destination is ses-Session2 type session
Group id: scien
Project label: BIDS_multi_session
container_path: scien/BIDS_multi_session/sub-TOME3024/ses-Session2
Creating script: BIDS_multi_session_session_ses-Session2.py ...



## Analysis Job, destination type is: subject

In [28]:
write_script_to_run_job(job_id="603fb4f225960896416e8ab6")

Job ID 603fb4f225960896416e8ab6
gear.gear.name is bids-app-template
job's destination_id is 603fb4f225960896416e8ab5 type analysis
job's analysis's parent id is 602ed7bcabe32939b783e906 type subject
new job's destination is sub-TOME3024 type subject
Group id: scien
Project label: BIDS_multi_session
container_path: scien/BIDS_multi_session/sub-TOME3024
Creating script: BIDS_multi_session_subject_sub-TOME3024.py ...

